In [9]:
from pandas_profiling import ProfileReport
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

import os
import time
import datetime
import urllib.request
import pytz

In [10]:
# fill next few rows with actual info
# by defalut, will be downloaded for Oslo, all existing data (since 2019-04 till 2022-06) into folder ./data.
city_name = 'oslo'        # можливі варіанти: olso, bergen, trondheim

start_month = 4           # 1 - 12
start_year = 2019         # 2019 - 2022

end_month = 6             # 1 - 12
end_year = 2022           # 2019 - 2022

destination_folder = '/Users/chedmurray/Desktop/безіменна папка 2'
# end filling

In [11]:
def to_date_str(date_arr):
    year = str(date_arr[0])
    month = ('0' + str(date_arr[1]))[-2:]
    return year + '/' + month

def add_month(date_str):
    date_arr = [int(i) for i in date_str.split('/')]
    if date_arr[1] != 12:
        date_arr[1] += 1
    else:
        date_arr[1] = 1
        date_arr[0] += 1
    return to_date_str(date_arr)

def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename, 'wb')
    file.write(response.read())
    file.close()

In [5]:
# generate all possible month-year combinations within the selected period
cur_date_str = to_date_str([start_year, start_month])
end_date_str = to_date_str([end_year, end_month])
dates_str_list = list()
if cur_date_str == end_date_str:
    dates_str_list = [cur_date_str]
else:
    while True:
        dates_str_list.append(cur_date_str)
        cur_date_str = add_month(cur_date_str)
        if cur_date_str == end_date_str:
            dates_str_list.append(cur_date_str)
            break

In [6]:
# download files
start = time.time()
if os.path.exists(destination_folder):
    for date in dates_str_list:
        download_url = f"https://data.urbansharing.com/{city_name}bysykkel.no/trips/v1/{date}.csv"
        filename = f"{city_name}_{date.replace('/', '_')}.csv"
        print("in progres: ", filename[:-4])
        destination_full_path = '/'.join([destination_folder, filename])
        download_file(download_url, destination_full_path)
else:
    print("Destination folder not found")
end = time.time()
print('Time: ', end - start, ' s')

in progres:  oslo_2019_04


KeyboardInterrupt: 

In [16]:
#dataset creating 

os.chdir("/Users/chedmurray/Desktop/безіменна папка 2") # working directory
csv = !ls

dataframes = (pd.read_csv(f) for f in csv)
df = pd.concat(dataframes, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659712 entries, 0 to 659711
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   started_at                 659712 non-null  object 
 1   ended_at                   659712 non-null  object 
 2   duration                   659712 non-null  int64  
 3   start_station_id           659712 non-null  int64  
 4   start_station_name         659712 non-null  object 
 5   start_station_description  659712 non-null  object 
 6   start_station_latitude     659712 non-null  float64
 7   start_station_longitude    659712 non-null  float64
 8   end_station_id             659712 non-null  int64  
 9   end_station_name           659712 non-null  object 
 10  end_station_description    659712 non-null  object 
 11  end_station_latitude       659712 non-null  float64
 12  end_station_longitude      659712 non-null  float64
dtypes: float64(4), int64(3), obje

In [17]:
# data transformation

# new timzone 
strptime_pattern = "%Y-%m-%d %H:%M:%S%z"
timezone = pytz.timezone('Europe/Oslo')

start = time.time()
df['started_at_dt'] = df['started_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
df['ended_at_dt'] = df['ended_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
end = time.time()
print("Трансформування: ", end - start, "с.")
df.head()

#add new columns 

df['day_of_week'] = df['started_at_dt'].dt.day_name()
df['month']=df['started_at_dt'].map(lambda x:x.month)
df['year']=df['started_at_dt'].map(lambda x:x.year)
df['hour']=df['started_at_dt'].map(lambda x:x.hour)
df['date']=df['started_at_dt'].map(lambda x:x.date())
df['for_count']=1

seasons = {(1, 12, 2): 'winter', (3, 4, 5): 'spring', (6, 7, 8): 'summer', (9, 10, 11): 'autumn'}

def season(ser):
    for k in seasons.keys():
        if ser in k:
            return seasons[k]

df['season'] = df['month'].apply(season)

df['duration'] = df['duration'].astype(int)

df.drop(labels=['started_at', 'ended_at'], axis=1, inplace=True)



Трансформування:  17.82568120956421 с.


In [18]:
# info by station about duration 

by_station_count_and_durationstats = df[['start_station_name', 'duration' ]].groupby(['start_station_name']).agg(
    number_of_trips=('duration', 'count'),
    min_duration=('duration', min),
    max_duration=('duration', max), 
    mean_duration=('duration', np.mean), 
    median_duration=('duration', np.median),
    standard_deviation_duration=('duration', np.std),
    variance_duration=('duration', np.var)).reset_index()

station_by_year_durationstats = df[['start_station_name', 'year', 'duration' ]].groupby(['start_station_name', 'year']).agg(
    number_of_trips=('duration', 'count'),
    min_duration=('duration', min),
    max_duration=('duration', max), 
    mean_duration=('duration', np.mean), 
    median_duration=('duration', np.median),
    standard_deviation_duration=('duration', np.std),
    variance_duration=('duration', np.var)).reset_index()

station_by_season_durationstats = df[['start_station_name', 'season', 'duration' ]].groupby(['start_station_name', 'season']).agg(
    number_of_trips=('duration', 'count'),
    min_duration=('duration', min),
    max_duration=('duration', max), 
    mean_duration=('duration', np.mean), 
    median_duration=('duration', np.median),
    standard_deviation_duration=('duration', np.std),
    variance_duration=('duration', np.var)).reset_index()

station_by_month_durationstats = df[['start_station_name', 'month', 'duration' ]].groupby(['start_station_name', 'month']).agg(
    number_of_trips=('duration', 'count'),
    min_duration=('duration', min),
    max_duration=('duration', max), 
    mean_duration=('duration', np.mean), 
    median_duration=('duration', np.median),
    standard_deviation_duration=('duration', np.std),
    variance_duration=('duration', np.var)).reset_index()

station_by_day_of_week_durationstats = df[['start_station_name', 'day_of_week', 'duration' ]].groupby(['start_station_name', 'day_of_week']).agg(
    number_of_trips=('duration', 'count'),
    min_duration=('duration', min),
    max_duration=('duration', max), 
    mean_duration=('duration', np.mean), 
    median_duration=('duration', np.median),
    standard_deviation_duration=('duration', np.std),
    variance_duration=('duration', np.var)).reset_index()

station_by_hour_durationstats = df[['start_station_name', 'hour', 'duration' ]].groupby(['start_station_name', 'hour']).agg(
    number_of_trips=('duration', 'count'),
    min_duration=('duration', min),
    max_duration=('duration', max), 
    mean_duration=('duration', np.mean), 
    median_duration=('duration', np.median),
    standard_deviation_duration=('duration', np.std),
    variance_duration=('duration', np.var)).reset_index()

In [19]:
station_by_year_trips = station_by_year_durationstats[['start_station_name', 
                                                       'number_of_trips' ]].describe().reset_index()

station_by_season = station_by_season_durationstats[['start_station_name', 
                                                     'number_of_trips' ]].groupby(['start_station_name']).describe().reset_index()

station_by_month = station_by_month_durationstats[['start_station_name',
                                                   'number_of_trips' ]].groupby(['start_station_name']).describe().reset_index()

station_by_day_of_week = station_by_day_of_week_durationstats[['start_station_name', 
                                                               'number_of_trips' ]].groupby(['start_station_name']).describe().reset_index()

station_by_hour = station_by_hour_durationstats[['start_station_name', 
                                                 'number_of_trips' ]].groupby(['start_station_name']).describe().reset_index()

In [21]:
station_by_season

start_station_name number_of_trips                              \
                                         count    mean         std     min   
0               7 Juni Plassen             2.0  1951.5   38.890873  1924.0   
1                          AHO             2.0  2451.5   13.435029  2442.0   
2                    Adamstuen             2.0   983.5   57.275649   943.0   
3                    Akersgata             2.0  2935.0   53.740115  2897.0   
4    Alexander Kiellands Plass             2.0  4733.5   91.216775  4669.0   
..                         ...             ...     ...         ...     ...   
243                     Volvat             2.0   517.5   12.020815   509.0   
244                     Vulkan             2.0  2450.5   85.559921  2390.0   
245                  Vålerenga             2.0   802.0   28.284271   782.0   
246  Vår Frelsers gravlund sør             2.0  3238.5   13.435029  3229.0   
247                 Økernveien             2.0   692.5  103.944697   619.0   

                                       
         25%     50%      75%     max  
0    1937.75  1951.5  1965.25  1979.0  
1    2446.75  2451.5  2456.25  2461.0  
2     963.25   983.5  1003.75  1024.0  
3    2916.00  2935.0  2954.00  2973.0  
4    4701.25  4733.5  4765.75  4798.0  
..       ...     ...      ...     ...  
243   513.25   517.5   521.75   526.0  
244  2420.25  2450.5  2480.75  2511.0  
245   792.00   802.0   812.00   822.0  
246  3233.75  3238.5  3243.25  3248.0  
247   655.75   692.5   729.25   766.0  

[248 rows x 9 columns]